In [1]:
import os
os.environ["PYTHONNET_RUNTIME"] = "coreclr"
os.environ["DOTNET_SYSTEM_DRAWING_USE_GDIPLUS"] = "1"

import clr
import numpy as np
import time
from datetime import timedelta
from pythonnet import load
from System import Environment, Array, String

# Load .NET Core runtime
load("coreclr")

# Try to import System.IO; fallback to os.chdir if it fails
try:
    from System.IO import Directory, Path, File
    use_dotnet_dir = True
except Exception as e:
    print(f"Note: System.IO import failed ({e}), using os.chdir instead")
    import os
    use_dotnet_dir = False

# Define DWSIM path
dwsimpath = "/usr/local/lib/dwsim/"

# Set working directory
if use_dotnet_dir:
    Directory.SetCurrentDirectory(dwsimpath)
else:
    os.chdir(dwsimpath)

# Add DWSIM assemblies
clr.AddReference(dwsimpath + "CapeOpen.dll")
clr.AddReference(dwsimpath + "DWSIM.Automation.dll")
clr.AddReference(dwsimpath + "DWSIM.Interfaces.dll")
clr.AddReference(dwsimpath + "DWSIM.GlobalSettings.dll")
clr.AddReference(dwsimpath + "DWSIM.SharedClasses.dll")
clr.AddReference(dwsimpath + "DWSIM.Thermodynamics.dll")
clr.AddReference(dwsimpath + "DWSIM.UnitOperations.dll")
clr.AddReference(dwsimpath + "DWSIM.Inspector.dll")
clr.AddReference(dwsimpath + "System.Buffers.dll")
clr.AddReference(dwsimpath + "DWSIM.Thermodynamics.ThermoC.dll")

# Now import DWSIM types
from DWSIM.Interfaces.Enums.GraphicObjects import ObjectType
from DWSIM.Thermodynamics import Streams, PropertyPackages
from DWSIM.UnitOperations import UnitOperations
from DWSIM.Automation import Automation3
from DWSIM.GlobalSettings import Settings

print("DWSIM imports successful!")

DWSIM imports successful!


In [2]:
# Create an instance of the Automation3 class from the DWSIM.Automation module
# This class provides methods for automating tasks in DWSIM, such as creating and manipulating flowsheets
interf = Automation3()

In [3]:
# Creates a flowsheet
sim = interf.CreateFlowsheet()

In [4]:
# Add Compounds
sim.AddCompound("Methane")
sim.AddCompound("Ethane")

In [5]:
one = sim.AddFlowsheetObject('Material Stream','1')
two = sim.AddFlowsheetObject('Material Stream','2')
three = sim.AddFlowsheetObject('Material Stream','3')
five = sim.AddFlowsheetObject('Material Stream','5')
six = sim.AddFlowsheetObject('Material Stream','6')
V_1 = sim.AddFlowsheetObject('Gas-Liquid Separator','V-1')
V_2 = sim.AddFlowsheetObject('Gas-Liquid Separator','V-2')
E1 = sim.AddFlowsheetObject('Energy Stream','E1')

In [6]:
one = one.GetAsObject()
two = two.GetAsObject()
three = three.GetAsObject()
five = five.GetAsObject()
six = six.GetAsObject()
V_1 = V_1.GetAsObject()
V_2 = V_2.GetAsObject()
E1 = E1.GetAsObject()

In [7]:
sim.ConnectObjects(one.GraphicObject , V_1.GraphicObject, -1, -1)
sim.ConnectObjects(V_1.GraphicObject , two.GraphicObject, -1, -1)
sim.ConnectObjects(V_1.GraphicObject , three.GraphicObject, -1, -1)
sim.ConnectObjects(three.GraphicObject , V_2.GraphicObject, -1, -1)
sim.ConnectObjects(V_2.GraphicObject , five.GraphicObject, -1, -1)
sim.ConnectObjects(V_2.GraphicObject , six.GraphicObject, -1, -1)
sim.ConnectObjects(E1.GraphicObject , V_2.GraphicObject, -1, -1)

In [8]:
sim.NaturalLayout()

In [9]:
one.SetOverallComposition(Array[float]([0.5,0.5]))
one.SetTemperature(300.0) # K
one.SetPressure(101325.0) # Pa
one.SetMassFlow(1.0) # kg/s

'1: mass flow set to 1 kg/s'

In [10]:
# property package
Thermo_Package = sim.CreateAndAddPropertyPackage("Peng-Robinson (PR)")

In [11]:
# Calc Modes
V_1.PressureCalculation.Minimum
V_2.PressureCalculation.Maximum
V_2.OverrideP = True
V_2.OverrideT = True
V_2.FlashTemperature = 298.15
V_2.FlashPressure = 101325

In [12]:
# request a calculation
errors = interf.CalculateFlowsheet4(sim)
list(errors)

[]

In [ ]:
# save file

fileNameToSave = Path.Combine(Environment.GetFolderPath(Environment.SpecialFolder.Desktop), "/workspace/00 FlowSheet Automation/02 Seperator Automation/02 Seperator.dwxmz")

interf.SaveFlowsheet(sim, fileNameToSave, True)